##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 애드온 옵티마이저: LazyAdam

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/optimizers_lazyadam"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/addons/tutorials/optimizers_lazyadam.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/addons/tutorials/optimizers_lazyadam.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/addons/tutorials/optimizers_lazyadam.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

## 개요

이 노트북은 애드온 패키지에서 게으른 adam 옵티마이저를 사용하는 방법을 보여줍니다.


## LazyAdam

> LazyAdam은 희소 업데이트를 보다 효율적으로 처리하는 Adam 옵티마이저의 변형입니다. 원래 Adam 알고리즘은 각 훈련 가능한 변수에 대해 두 개의 이동 평균 누산기(moving-average accumulator)를 유지합니다. 누산기는 모든 단계에서 업데이트됩니다. 이 클래스는 희소 변수에 대한 그래디언트 업데이트의 지연 처리를 제공합니다. 모든 인덱스에 대한 누산기를 업데이트하지 않고 현재 배치에 나타나는 희소 변수 인덱스에 대한 이동 평균 누산기만 업데이트합니다. 원래 Adam 옵티마이저와 비교하여 일부 애플리케이션의 모델 훈련 처리량을 크게 향상할 수 있습니다. 그러나 원래 Adam 알고리즘과 약간 다른 의미를 제공하며 경험적 결과가 다를 수 있습니다.

## 설정

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
# Hyperparameters
batch_size=64
epochs=10

## 모델 빌드하기

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(784,), activation='relu', name='dense_1'),
    tf.keras.layers.Dense(64, activation='relu', name='dense_2'),
    tf.keras.layers.Dense(10, activation='softmax', name='predictions'),
])

## 데이터 준비하기

In [ ]:
# Load MNIST dataset as NumPy arrays
dataset = {}
num_validation = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

## 훈련 및 평가하기

일반적인 keras 옵티마이저를 새로운 tfa 옵티마이저로 간단히 교체합니다. 

In [ ]:
# Compile the model
model.compile(
    optimizer=tfa.optimizers.LazyAdam(0.001),  # Utilize TFA optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs)


In [ ]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))